In [1]:
#import modules
%run ../../load_main.py

# Get linearity coefficients

In [2]:
# pm25

In [3]:
pm25=pd.read_excel('pm25_linear.xlsx').drop('Unnamed: 0',axis=1)

In [4]:
pm25.head()

,sectors,scale_factor,value
0,NCT_DOM,100,92.714272
1,NCT_IPO,100,92.714272
2,NCT_TRA,100,92.714272
3,NCR_DOM,100,92.714272
4,NCR_IPO,100,92.714272


In [5]:
# change types data
pm25.sectors = pm25.sectors.astype(str)
pm25.scale_factor = pm25.scale_factor.astype(float)/100

In [6]:
# get boundaries of linaerity
b_inf = 0.5
b_upp = 1.5

pm25_lin=pm25[ ( pm25['scale_factor'] >= b_inf ) & ( pm25['scale_factor'] <= b_upp) ]

In [7]:
df=pd.DataFrame(0.000,columns=['slope_pm25','slope_o3'],index=pm25.sectors.unique())

In [8]:
df

,slope_pm25,slope_o3
NCT_DOM,0.0,0.0
NCT_IPO,0.0,0.0
NCT_TRA,0.0,0.0
NCR_DOM,0.0,0.0
NCR_IPO,0.0,0.0
NCR_TRA,0.0,0.0
ALL,0.0,0.0


In [9]:
import scipy.stats as ss
# for each secto caluclate the slope and put in dataframe
for s in pm25_lin.sectors.unique():
    t=pm25_lin[pm25_lin.sectors==s]
    
    x=t.scale_factor.values
    y=t.value.values
    interp =ss.linregress(x, y)
    
    df.at[s,'slope_pm25']=interp.slope # % change

In [10]:
df

,slope_pm25,slope_o3
NCT_DOM,1.655382,0.0
NCT_IPO,5.262512,0.0
NCT_TRA,8.748517,0.0
NCR_DOM,10.601917,0.0
NCR_IPO,12.179391,0.0
NCR_TRA,6.937451,0.0
ALL,45.413290,0.0


In [11]:
# ozone

In [12]:
o3=pd.read_excel('o3DM8H_linear.xlsx').drop('Unnamed: 0',axis=1)

In [13]:
o3.head()

,sectors,scale_factor,value
0,NCT_DOM,100,82.106865
1,NCT_IPO,100,82.106865
2,NCT_TRA,100,82.106865
3,NCR_DOM,100,82.106865
4,NCR_IPO,100,82.106865


In [14]:
# change types data
o3.sectors = o3.sectors.astype(str)
o3.scale_factor = o3.scale_factor.astype(float)/100

In [15]:
# get boundaries of linaerity
o3_lin=o3[ ( o3['scale_factor'] >= b_inf ) & ( o3['scale_factor'] <= b_upp) ]

In [16]:
# for each secto caluclate the slope and put in dataframe
for s in o3_lin.sectors.unique():
    t=o3_lin[o3_lin.sectors==s]
    
    x=t.scale_factor.values
    y=t.value.values
    interp =ss.linregress(x, y)
    
    df.at[s,'slope_o3']=interp.slope

In [17]:
df

,slope_pm25,slope_o3
NCT_DOM,1.655382,-0.734131
NCT_IPO,5.262512,-3.839755
NCT_TRA,8.748517,-27.914084
NCR_DOM,10.601917,-1.059229
NCR_IPO,12.179391,-9.808801
NCR_TRA,6.937451,-21.571664
ALL,45.413290,-65.617172


In [18]:
df.to_excel('slopes.xlsx')

# Calculate mortality

In [19]:
# RR2 for PM2.5: Eq. 4 from paper:
#https://www.pnas.org/content/117/41/25370
import math
rmin=2.4
def rr_p(x):
    y=math.exp((0.25*math.log((x-rmin)/6.5+1))/(1+math.exp(-1*(x-rmin-2.5)/32))) #y= math.sin(x) #
    return y

In [20]:
# RR for O3: Eq. 5 from paper:
#https://www.pnas.org/content/117/41/25370
def rr_o(x,c0=20,val=1.03):
    #f=(x/((x-80)*2))
    if x<=c0:
        f=1
    else:
        f=pow(val,(x-c0)/(10*2))  # original is /10 (ppb) we have ug m-3 so we convert roughly to /20 ug m-3
        
    return f

In [21]:
# constants
pm25_base=92.714272
o3_base=82.106865
p =  31e6 # delhi population in 2020 according to UN World Urbanization Prospects 2018 https://population.un.org/wup/DataQuery/
I = 53.9/1e5 # for COPD in Delhi from https://www.sciencedirect.com/science/article/pii/S0160412016300848?via%3Dihub#s0045

In [22]:
# functions

def delta_pm(x):
    delta_pm25 =  (      df.at['NCT_IPO','slope_pm25']* (x[0]-1) +
                         df.at['NCT_DOM','slope_pm25']* (x[1]-1) +
                         df.at['NCT_TRA','slope_pm25']* (x[2]-1) +
                         df.at['NCR_IPO','slope_pm25']* (x[3]-1) +
                         df.at['NCR_DOM','slope_pm25']* (x[4]-1) +
                         df.at['NCR_TRA','slope_pm25']* (x[5]-1)
                        )
    
    return delta_pm25


def delta_o3(x):
    delta_o3 = (df.at['NCT_IPO','slope_o3']* (x[0]-1) +
                     df.at['NCT_DOM','slope_o3']* (x[1]-1) +
                     df.at['NCT_TRA','slope_o3']* (x[2]-1) +
                     df.at['NCR_IPO','slope_o3']* (x[3]-1) +
                     df.at['NCR_DOM','slope_o3']* (x[4]-1) +
                     df.at['NCR_TRA','slope_o3']* (x[5]-1)
                    )
                        
    return delta_o3


def mortality_pm(p,I,c):
     return p*I*(1-1/rr_p(c))
    

def mortality_o3(p,I,c):
     return p*I*(1-1/rr_o(c))


    
def delta_mortality(x): 
    arg_pm25 = 1/rr_p(pm25_base+delta_pm(x)) - 1/rr_p(pm25_base)
    
    
    arg_o3 = 1/rr_o(o3_base+delta_o3(x)) - 1/rr_o(o3_base)
                            
    return -1*p*I*(arg_pm25+arg_o3)     # (max function = min -function)

In [23]:
# OPTIMIZE

from scipy.optimize import differential_evolution, minimize
import numpy as np

b_inf = 0.5
b_upp = 1
bounds = [(b_inf,b_upp)]*6


# try two different algorithms
result = differential_evolution(delta_mortality, bounds)  

results2=minimize(delta_mortality,[1]*6, bounds=bounds)

In [24]:
result

     fun: -371.3029810732499
     jac: array([ 133.89239371,   52.66863354, -256.97918318,  289.33570775,
        418.44721181, -191.22282993])
 message: 'Optimization terminated successfully.'
    nfev: 4064
     nit: 44
 success: True
       x: array([0.5, 0.5, 1. , 0.5, 0.5, 1. ])

In [25]:
results2

      fun: -371.3029810732499
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 133.89239371,   52.66863354, -256.97918318,  289.33570775,
        418.44721181, -191.22282993])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 14
      nit: 1
   status: 0
  success: True
        x: array([0.5, 0.5, 1. , 0.5, 0.5, 1. ])

results say to target max reduction of DOM, and IPO, NCR_IPO, not TRA 
probably because TRA and NCR_TRA have the steepest O3 increase when reducing emissions comapred to the others.

In [26]:
result

     fun: -371.3029810732499
     jac: array([ 133.89239371,   52.66863354, -256.97918318,  289.33570775,
        418.44721181, -191.22282993])
 message: 'Optimization terminated successfully.'
    nfev: 4064
     nit: 44
 success: True
       x: array([0.5, 0.5, 1. , 0.5, 0.5, 1. ])

In [27]:
results2

      fun: -371.3029810732499
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 133.89239371,   52.66863354, -256.97918318,  289.33570775,
        418.44721181, -191.22282993])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 14
      nit: 1
   status: 0
  success: True
        x: array([0.5, 0.5, 1. , 0.5, 0.5, 1. ])

In [28]:
# baseline mortality due to PM2.5 and O3

In [29]:
m_base = mortality_o3(p,I,o3_base)+ mortality_pm(p,I,pm25_base)

In [30]:
m_base

9314.898687284787

In [31]:
result.fun  # number of avoided deaths

-371.3029810732499

In [32]:
abs(result.fun)/365  # avoided deaths per day

1.0172684412965751

In [33]:
abs(result.fun)/365*15  # total avoided deaths in 15 days

15.259026619448626

In [34]:
result.fun/m_base*100      # pecent of total avoidavble deaths

-3.9861193721848363